# Tutorial - how to work with large genomic flat files using `tabix`

In bioinformatics today we typically have to work with large scale genomic datasets that are too large to enter into relational databases. T allow efficient indexing and searching of compressed genomic file types the `tabix` tool is widely used.

To demonstrate this we need some data. In the directory tabix_tutorial is a directory called VCFs. This contains two small VCF files to complete this tutorial with. The vcf files have been compressed with the `bgzip` utility. We also have a file containing regions we want to extract from the VCF file.

In [ ]:
%%!
ls tabix_tutorial/VCFs

To use the tabix command we need to use a package called `bcftools` which contains the HTSlib utility. HTSlib is also available as a standalone package, and ideally we should use `tabix` within bcftools as it means we can use the same package for our entire analysis. However, in the version available to install via conda for some reason it is not bundled with bcftools. Install them both on your system with conda using the following:

In [ ]:
%%!
conda install -c bioconda bcftools

In [ ]:
%%!
conda install -c bioconda tabix

To use tabix we need to work with the compressed genomic format generated by bgzip. Note that the extension is gz. However, you must use bgzip and not gzip to compress. `bgzip` is part of `tabix`, so now we have installed `tabix` we can practice this on the one vcf file in your folder that is not compressed. Replace the file name below with the name of the uncompressed file in your VCFs directory.

In [ ]:
%%!
bgzip <vcf_name.vcf>

Now we can play around with tabix to see what it does. The first thing we need to do is to use `tabix` to index our vcf.gz files. The following command will index a compressed vcf file. <br>
*In the following examples I have included a completed code chunk underneath the example*

In [ ]:
%%!
tabix -p vcf <vcf_name.vcf.gz>

e.g.

In [ ]:
%%!
tabix -p vcf tabix_tutorial/VCFs/sample_a.vcf.gz

Now let's choose a genomic region to look at. I know there is a difference between these samples on the long arm of chromosome 3. It is easiest in `tabix` to provide regions in a file. I call this regions.txt.

In [ ]:
%%!
head tabix_tutorial/regions.txt

We can then extract anything within the regions specified in the file regions.txt. To keep the file in VCF format we can also use -h to add the VCF header to the output file.

In [ ]:
%%!
tabix -h <vcf_name.vcf.gz> -R regions.txt >> tabix_results.txt

e.g.

In [ ]:
%%!
tabix -h tabix_tutorial/VCFs/sample_a.vcf.gz -R tabix_tutorial/regions.txt >> tabix_results.txt

This is an efficient way to query a flat file database of variants in a compressed format. But typically VCF files only represent a database problem in large cohort studies where we have 100,000+ 90GB VCF files that we want to query for the same region. How can we manage that? The answer is in two stages. First, we need a way to print the lines of a VCF file and associate this with a sample. We can achieve this by piping the output from `tabix` into `bcftools`.

In [ ]:
%%!
tabix -h <vcf_name.vcf.gz> -R regions.txt | bcftools query -f '[%SAMPLE]\t%CHROM\t%POS\t%ID\t%REF\t%ALT\t%QUAL\t%FILTER\t[%GT]\t[%GQ]\t[%DP]\n' >> bcftools_results.txt

e.g. (ignore the warning messages)

In [ ]:
%%!
tabix -h tabix_tutorial/VCFs/sample_a.vcf.gz -R tabix_tutorial/regions.txt | bcftools query -f '[%SAMPLE]\t%CHROM\t%POS\t%ID\t%REF\t%ALT\t%QUAL\t%FILTER\t[%GT]\t[%GQ]\t[%DP]\n' >> bcftools_results.txt

Now we have an output where our sample ID is in a seperate column. We could put this anywhere within the record, but let's kep it at the start. Next, we need to loop this operation for multiple samples. For ease let's put these in a seperate file (vcf_list.txt). Make yourself a file containing the paths to the VCF files. You will need to add your new vcf.gz file to the list. You will need to complete the path to wherever the directory Tutorial_5 is located.

In [ ]:
%%writefile vcf_list.txt
/tabix_tutorial/VCFs/sample_a.vcf.gz
/tabix_tutorial/VCFs/sample_b.vcf.gz
/tabix_tutorial/VCFs/sample_c.vcf.gz

Now we can put our command in a shell script, use a while loop to cycle through our vcf_list.txt file, and print everything out to the same file.

In [ ]:
%%writefile query.sh
######################## Extract variants in a cohort by chromosomal cooordinates ###########################################
# This script subsets VCFs by coordinate and prints out sample ID and genotype

# INPUT:
        # 1: Tab-delimited list of chromosomal coordinates (regions.txt)
        # 2. A list of full-paths to VCF files of interest (vcf_list.txt)

# OUTPUT:
        # 1: A tab-delimted file of sample ID, variant information and quality (results.txt)

# STEPS:
        # 1: tabix to subset each VCF by the coordinates given
        # 2: bcftools to print out the sample ID, and variant and quality information as a text file

#!/bin/bash

while read -r vcf; do
        tabix -h $vcf -R regions.txt | \
        bcftools query -f '[%SAMPLE]\t%CHROM\t%POS\t%ID\t%REF\t%ALT\t%QUAL\t%FILTER\t[%GT]\t[%GQ]\t[%DP]\n' >> cohort_results.txt;
done < vcf_list.txt

sed -i '1s/^/SAMPLE\tCHROM\tPOS\tID\tREF\tALT\tQUAL\tFILTER\tGT\tGQ\tDP\n/' cohort_results.txt

In [ ]:
%%!
chmod +rx query.sh
./query.sh

Have a look at your output file:

In [ ]:
%%!
head cohort_results.txt

**Over to you:** Install bcftools and ensure that you can get the while loop to work. Make tabix indexes for each vcf.gz file and compress the remaining .vcf file. After running the script on the discussion board tell me if our four VCF files contained any variants in the genomic region specified in regions.txt and if there were any variants that were unique to samples a and b or c and d. Samples a and b are control samples and c and d are mutant samples.

By doing this exercise you will have utilised the most common index tool for genomic data formats. This is important because it makes you consider the special properties of Big Data, and how the solutions we have to deal with them do not always match what might be expected. Here we have set-up a file store as a database and queried it using bash. This is rudimentary - but highlights that good commandline usage is not going anywhere fast!